[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ciandorr/truthmakers/blob/main/truthmaker_viz.ipynb)

# Visualizing Kit Fine's Regular Bilateral Truthmaker Semantics

This notebook provides an interactive environment for exploring bilateral regular propositions in Kit Fine's truthmaker semantics.

## What is Truthmaker Semantics?

In bilateral truthmaker semantics:
- **States** form a complete semilattice.  The GLB of a set of states is called their *fusion*.  
- **Propositions** can be represented as pairs of a set of **verifiers** (the states that make the proposition true) and a set of
**falsifiers** (the states that make it false).

In *regular* bilateral truthmaker semantics, the verifier and falsifier sets are required to be *convex* (any state between two members is a member) and *closed* (the fusion of any set of members is a member).  

This notebook lets you:
1. Build an arbitrary finite semilattice to be the set of states.
2. Construct a set of propositions on this set of states, which may be closed under any and all of the logical operations of negation, conjunction, and disjunction.
4. Visualize the structure of this set of propositions, in several different ways.

## 1. Setup

Run the cells below to set up the environment. On Google Colab, the first cell will clone the repository and install dependencies.

In [ ]:
# Google Colab setup (no-op when running locally)
import os
IN_COLAB = 'COLAB_RELEASE_TAG' in os.environ

if IN_COLAB:
    if not os.path.exists('/content/truthmakers/truthmaker_core.py'):
        !git clone https://github.com/ciandorr/truthmakers.git /content/truthmakers
    os.chdir('/content/truthmakers')
    !pip install -q -r requirements.txt
    print("Colab setup complete.")
else:
    print("Running locally.")

In [ ]:
# Ensure IN_COLAB is defined (in case setup cell was skipped)
try:
    IN_COLAB
except NameError:
    IN_COLAB = False

# Import core modules
import sys
sys.path.insert(0, '.')

# Force reload to pick up any changes
import importlib
import truthmaker_core
import truthmaker_visualization
import truthmaker_notebook_apps
importlib.reload(truthmaker_core)
importlib.reload(truthmaker_visualization)
importlib.reload(truthmaker_notebook_apps)

from truthmaker_core import (
    State, RegularSet, BilateralProposition,
    PropositionSet, TruthmakerPoset,
    parse_propositions, definites, diamonds,
    L_reorganize, M_reorganize,
    has_null_m_class,
    StateSpace, get_state_space, set_state_space, reset_state_space
)

from truthmaker_visualization import create_plotly_graph, export_to_dot, export_to_csv

from truthmaker_notebook_apps import (
    create_proposition_builder_app,
    create_dual_hasse_app,
    bilateral_to_prop_dict,
    prop_dict_to_bilateral,
    prop_list_to_proposition_set,
    deserialize_regular_set,
    serialize_regular_set
)

import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, HTML
import networkx as nx

# =============================================================================
# GLOBAL: The live proposition list (shared across all cells)
# =============================================================================
PROPOSITION_LIST = []

def parse_with_commas(prop_strings):
    """Parse propositions, converting comma-separated states to space-separated."""
    converted = []
    for s in prop_strings:
        parts = s.split('|')
        converted_parts = []
        for part in parts:
            subparts = part.split(';')
            if len(subparts) >= 1:
                subparts[0] = subparts[0].replace(',', ' ')
            converted_parts.append(';'.join(subparts))
        converted.append('|'.join(converted_parts))
    return parse_propositions(converted)

def run_dash_app(app, port, height=540):
    """Run a Dash app with environment-appropriate display."""
    if IN_COLAB:
        import threading, time
        def start_server():
            try:
                app.run(port=port, host='127.0.0.1', debug=False)
            except OSError:
                pass  # Port already in use from previous run
        threading.Thread(target=start_server, daemon=True).start()
        time.sleep(2)
        from google.colab import output
        output.serve_kernel_port_as_iframe(port, height=height)
    else:
        app.run(
            jupyter_mode='inline', jupyter_height=height,
            jupyter_width='100%', debug=False, port=port
        )

# Auto-apply default state space (rgb)
default_space = StateSpace.from_atoms('rgb', include_empty=False)
set_state_space(default_space)

print("Modules loaded successfully")
print(f"Default state space: {sorted(default_space.all_states())}")

# 2. State Space Configuration

The state space is the underlying join-semilattice of states.  You can skip this cell if you want to use the default; otherwise, you have three options for building a custom state space. 

**Options:**
- **Auto-generate**: Nonempty subsets of a given list of "atomic" states (e.g., "rgb" → r, g, b, rg, rb, gb, rgb)
- **Auto-generate with empty state**: The same, but adding ∅ as the bottom element
- **Custom**: Define your own states and ≤ relation: the parthood relation on the state space will be the transitive closure of this ≤.  

Click **"Apply State Space"** to set up your state space.

In [2]:
# State Space Configuration
# -------------------------
# Mode selector
mode_selector = widgets.RadioButtons(
    options=[
        ('Auto-generate from atoms (default)', 'auto'),
        ('Auto-generate with empty state (-)', 'auto_empty'),
        ('Custom state space', 'custom')
    ],
    value='auto',
    description='Mode:',
    style={'description_width': '60px'}
)

# Atom input for auto mode
atom_input = widgets.Text(
    value='rgb',
    description='Atoms:',
    placeholder='e.g., rgb or rgby',
    style={'description_width': '60px'},
    layout=widgets.Layout(width='300px')
)

# Custom states input
custom_states_input = widgets.Textarea(
    value='a\nb\nab',
    description='States:',
    placeholder='One state name per line',
    layout=widgets.Layout(width='300px', height='80px'),
    style={'description_width': '60px'}
)

# Custom relations input
custom_relations_input = widgets.Textarea(
    value='a ≤ ab\nb ≤ ab',
    description='Relations:',
    placeholder='e.g., a ≤ ab (one per line)',
    layout=widgets.Layout(width='300px', height='80px'),
    style={'description_width': '60px'}
)

# Apply button
apply_ss_button = widgets.Button(
    description='Apply State Space',
    button_style='primary',
    icon='check'
)

# Output areas
ss_output = widgets.Output()
hasse_output = widgets.Output()

def update_visibility(change):
    """Show/hide inputs based on mode."""
    mode = change['new']
    if mode == 'custom':
        atom_input.layout.display = 'none'
        custom_states_input.layout.display = ''
        custom_relations_input.layout.display = ''
    else:
        atom_input.layout.display = ''
        custom_states_input.layout.display = 'none'
        custom_relations_input.layout.display = 'none'

mode_selector.observe(update_visibility, names='value')

def show_hasse_diagram(space):
    """Display Hasse diagram of state space."""
    with hasse_output:
        hasse_output.clear_output()
        
        edges = space.hasse_edges()
        G = nx.DiGraph()
        G.add_nodes_from(space.all_states())
        G.add_edges_from(edges)
        
        # Use hierarchical layout
        try:
            from truthmaker_visualization import compute_layout
            pos = compute_layout(G, 'hierarchical')
        except:
            pos = nx.spring_layout(G, k=2, iterations=50)
        
        # Create Plotly figure
        edge_x, edge_y = [], []
        for e in edges:
            x0, y0 = pos[e[0]]
            x1, y1 = pos[e[1]]
            edge_x.extend([x0, x1, None])
            edge_y.extend([y0, y1, None])
        
        node_x = [pos[n][0] for n in G.nodes()]
        node_y = [pos[n][1] for n in G.nodes()]
        node_text = list(G.nodes())
        
        fig = go.Figure(data=[
            go.Scatter(x=edge_x, y=edge_y, mode='lines',
                      line=dict(width=1, color='gray'), hoverinfo='none'),
            go.Scatter(x=node_x, y=node_y, mode='markers+text',
                      marker=dict(size=25, color='lightyellow', line=dict(width=2, color='gray')),
                      text=node_text, textposition='middle center',
                      textfont=dict(size=10),
                      hoverinfo='text')
        ])
        
        fig.update_layout(
            title=f'State Space ({len(space.all_states())} states)',
            showlegend=False, width=500, height=400,
            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            plot_bgcolor='white'
        )
        display(fig)

def on_apply_ss_click(b):
    global PROPOSITION_LIST
    with ss_output:
        ss_output.clear_output()
        hasse_output.clear_output()
        mode = mode_selector.value
        
        try:
            if mode == 'auto':
                space = StateSpace.from_atoms(atom_input.value, include_empty=False)
                print(f"✓ Generated {len(space.all_states())} states from atoms '{atom_input.value}'")
                print(f"  States: {sorted(space.all_states())}")
            elif mode == 'auto_empty':
                space = StateSpace.from_atoms(atom_input.value, include_empty=True)
                print(f"✓ Generated {len(space.all_states())} states (including -)")
                print(f"  States: {sorted(space.all_states())}")
            else:
                # Parse custom input
                states = [s.strip() for s in custom_states_input.value.split('\n') if s.strip()]
                relations = []
                for line in custom_relations_input.value.split('\n'):
                    line = line.strip()
                    if '≤' in line or '<=' in line:
                        parts = line.replace('<=', '≤').split('≤')
                        if len(parts) == 2:
                            relations.append((parts[0].strip(), parts[1].strip()))
                
                space = StateSpace.from_relations(states, relations)
                print(f"✓ Created custom state space with {len(space.all_states())} states")
                
                # Show added states from auto-completion
                original = set(states)
                added = space.all_states() - original
                if added:
                    print(f"  Added {len(added)} states for semilattice completion: {added}")
            
            # Validate
            errors = space.validate()
            if errors:
                for err in errors[:5]:
                    print(f"⚠ {err}")
            
            # Apply globally
            set_state_space(space)
            print("\n✓ State space applied globally")
            
            # Clear proposition list when state space changes
            PROPOSITION_LIST = []
            print("  (Proposition list cleared - rebuild in Section 2)")
            
            # Show Hasse diagram for non-trivial spaces
            if len(space.all_states()) <= 20:
                show_hasse_diagram(space)
            else:
                print(f"\n(Hasse diagram skipped - {len(space.all_states())} states is too many to display)")
                
        except Exception as e:
            print(f"✗ Error: {e}")
            import traceback
            traceback.print_exc()

apply_ss_button.on_click(on_apply_ss_click)

# Initial visibility
update_visibility({'new': 'auto'})

# Layout
display(widgets.VBox([
    widgets.HTML("<b>State Space Configuration</b>"),
    mode_selector,
    atom_input,
    custom_states_input,
    custom_relations_input,
    apply_ss_button
]))
display(ss_output)
display(hasse_output)

print("Configure your state space and click 'Apply State Space'.")

Output()

Output()

Configure your state space and click 'Apply State Space'.


## 2. Build Proposition List

Use the **Visual Proposition Builder** below to create and manage your propositions.

**Controls:**
- **Auto-Init**: Generate default atom propositions (one per atom, mutually exclusive)
- **Add from text**: Parse propositions from text format
- **Add New**: Create an empty proposition and build it by clicking states

**Node colors:** Green = Verifier | Red = Falsifier | Yellow = Both | Gray = Neither

**Closure operations:** Close the list under NOT, AND, OR, or all three

**Important:** If you change the state space, you must re-run the builder cell to refresh.

In [ ]:
# =============================================================================
# EXAMPLE PROPOSITION STRINGS (for copy/paste into "Add from text")
# =============================================================================
# These use comma-separated states and format: antichain;top|antichain;top
#
# atoms1 (3 mutually exclusive):
#   r;r|g,b;gb
#   g;g|r,b;rb
#   b;b|r,g;rg
#
# atoms2 (upward closed verifiers/falsifiers):
#   r;rgb|g,b;rgb
#   g;rgb|r,b;rgb
#   b;rgb|r,g;rgb
#
# atoms3 (singleton verifiers, fused falsifiers):
#   r;r|gb;gb
#   g;g|rb;rb
#   b;b|rg;rg
#
# atoms4 (upward closed alternative):
#   r;rgb|gb;rgb
#   g;rgb|rb;rgb
#   b;rgb|rg;rgb
#
# atoms5 (2 compossible atoms):
#   r;r|b;b
#   g;g|b;b
#   b;b|r,g;rg
#
# atoms6 (4 mutually exclusive, requires 'rgby' state space):
#   r;r|g,b,y;gby
#   g;g|r,b,y;rby
#   b;b|r,g,y;rgy
#   y;y|r,g,b;rgb
# =============================================================================

# Run the proposition builder
space = get_state_space()
print(f"State space: {sorted(space.all_states())}")
print(f"Current propositions: {len(PROPOSITION_LIST)}")

builder_app = create_proposition_builder_app(PROPOSITION_LIST)
run_dash_app(builder_app, port=8052, height=540)

## 3. Visualize Proposition Poset

Visualize the list of propositions from the **Proposition Builder** (Section 2) as a partially ordered set, ordered by conjunctive (q = p ∧ q) or disjunctive (q = p ∨ q) parthod.

In [4]:
# Poset Build & Visualize - reads from PROPOSITION_LIST

# Build controls
order_type_dropdown = widgets.Dropdown(
    options=['conjunction', 'disjunction'],
    value='conjunction',
    description='Order type:',
    style={'description_width': '100px'}
)

reduce_checkbox = widgets.Checkbox(value=True, description='Transitive reduction')

# Visualization controls
layout_dropdown = widgets.Dropdown(
    options=['hierarchical', 'force', 'circular'],
    value='hierarchical',
    description='Layout:',
    style={'description_width': '100px'}
)

color_dropdown = widgets.Dropdown(
    options=['generation', 'definite', 'diamond', 'uniform'],
    value='generation',
    description='Color by:',
    style={'description_width': '100px'}
)

show_edges_checkbox = widgets.Checkbox(value=True, description='Show edges')

visualize_button = widgets.Button(description='Build & Visualize', button_style='success', icon='eye')

viz_output = widgets.Output()

# Global for other tools
poset = None

def on_visualize_click(b):
    global poset
    with viz_output:
        viz_output.clear_output(wait=True)

        prop_set = prop_list_to_proposition_set(PROPOSITION_LIST)
        if prop_set is None or len(prop_set) == 0:
            print("No propositions in builder!")
            print("Use the Proposition Builder above to add and optionally close propositions.")
            return

        print(f"Building {order_type_dropdown.value} poset from {len(prop_set)} propositions...")
        poset = TruthmakerPoset(prop_set, order_type=order_type_dropdown.value)

        if reduce_checkbox.value:
            poset = poset.transitive_reduction()

        print(f"Poset: {len(poset.prop_list)} nodes, {poset.graph.number_of_edges()} edges")
        print("Generating visualization...")

        fig = create_plotly_graph(
            poset,
            layout=layout_dropdown.value,
            color_by=color_dropdown.value,
            show_edges=show_edges_checkbox.value
        )

        viz_output.clear_output()
        display(fig)

visualize_button.on_click(on_visualize_click)

display(widgets.VBox([
    widgets.HTML("<b>Poset Options:</b>"),
    widgets.HBox([order_type_dropdown, reduce_checkbox]),
    widgets.HTML("<br><b>Visualization Options:</b>"),
    widgets.HBox([layout_dropdown, color_dropdown, show_edges_checkbox]),
    visualize_button
]))
display(viz_output)

Output()

## 4. Dual Hasse Diagram (Interactive)

Where p and q are propositions, we say that p ≤_L q (p "logically entails" q) iff p ∨ (p ∧ q) = p ∨ q, and p ≡_L q (p is "logically equivalent" to q) iff p ≤_L q and q ≤_L p.  

We can also define a notion of "nonlogical" containment: p ≤_M q iff q = (p ∧ q) ∨ q; or alternatively, p ≤_M q iff q = (p ∨ q) ∧ q.  (These two definitions can diverge only when one of the two propositions has an empty verifier or falsifier set.)  Similarly, p ≡_M q iff p ≤_M q and q ≤_M p.  A basic property of the regular semantics is "two-dimensionality": if p ≡_L q and p ≡_M q, then p = q.  

This visualization presents two side-by side Hasse diagrams, which represent, respectively, the ≡_L-equivalence classes of the propositions from the **Proposiion Builder** ordered by ≤_L, and the ≡_M equivalence classes of these propositions ordered by ≤_M.  Each proposition is represented as a red line connecting its L-class and its M-class.  

**Features:**
- Hover over an L-class to see a corresponding bilateral proposition with upward-closed verifiers and falsifiers.
- Hover over an M-class to see a corresponding bilateral proposition with downward-closed verifiers and falsifiers (and a unique top element).  
- Click on a node to see all the propositions contained in the corresponding class.  

In [ ]:
# Create and run the dual Hasse diagram from PROPOSITION_LIST

prop_set = prop_list_to_proposition_set(PROPOSITION_LIST)

if prop_set is None or len(prop_set) == 0:
    print("No propositions in builder!")
    print("Use the Proposition Builder above to add and optionally close propositions.")
else:
    print(f"Found {len(prop_set)} propositions.")

    # Check if any propositions have null verifiers/falsifiers
    has_nulls = has_null_m_class(prop_set)

    if has_nulls:
        print("\nSome propositions have null verifiers or falsifiers.")
        print("This creates ambiguity in the M-class ordering.\n")

        ordering_dropdown = widgets.Dropdown(
            options=[
                ('Option A: null verifiers at bottom, null falsifiers at top', 'A'),
                ('Option B: null verifiers at top, null falsifiers at bottom', 'B')
            ],
            value='A',
            description='M-ordering:',
            style={'description_width': '100px'},
            layout=widgets.Layout(width='500px')
        )

        run_button = widgets.Button(description='Create Dual Hasse Diagram', button_style='success', icon='play')
        output_area = widgets.Output()

        def on_run_click(b):
            with output_area:
                output_area.clear_output()
                m_ordering = ordering_dropdown.value
                print(f"Creating dual Hasse diagram with M-ordering option {m_ordering}...")
                dual_app = create_dual_hasse_app(prop_set, m_ordering=m_ordering)
                run_dash_app(dual_app, port=8051, height=750)

        run_button.on_click(on_run_click)

        display(widgets.HTML("<b>Choose M-class ordering for null propositions:</b>"))
        display(widgets.VBox([ordering_dropdown, run_button]))
        display(output_area)
    else:
        print("Creating dual Hasse diagram...")
        dual_app = create_dual_hasse_app(prop_set)
        run_dash_app(dual_app, port=8051, height=750)

## 5. Exploration Tools

Explore specific nodes and query the proposition set.

In [ ]:
# Node inspector
node_selector = widgets.IntText(value=0, description='Node ID:', style={'description_width': '100px'})
inspect_button = widgets.Button(description='Inspect Node', button_style='info', icon='search')
inspect_output = widgets.Output()

def on_inspect_click(b):
    with inspect_output:
        inspect_output.clear_output()

        if poset is None:
            print("Please build poset first!")
            return

        node_id = node_selector.value
        if node_id < 0 or node_id >= len(poset.prop_list):
            print(f"Invalid node ID. Must be 0-{len(poset.prop_list)-1}")
            return

        prop = poset.get_proposition(node_id)

        print(f"NODE {node_id}")
        print("=" * 60)
        print(f"Proposition: {prop.to_string()}\n")

        print("VERIFIERS:")
        if prop.verifiers.is_null:
            print("  (null)")
        else:
            print(f"  Antichain: {{{', '.join([s.to_string() for s in prop.verifiers.antichain])}}}")
            print(f"  Top state: {prop.verifiers.top_state.to_string()}")

        print("\nFALSIFIERS:")
        if prop.falsifiers.is_null:
            print("  (null)")
        else:
            print(f"  Antichain: {{{', '.join([s.to_string() for s in prop.falsifiers.antichain])}}}")
            print(f"  Top state: {prop.falsifiers.top_state.to_string()}")

        print("\nPROPERTIES:")
        print(f"  Definite: {prop.is_definite()}")
        print(f"  Diamond: {prop.is_diamond()}")

        print("\nPOSET RELATIONS:")
        preds = poset.predecessors(node_id)
        succs = poset.successors(node_id)
        print(f"  Predecessors ({len(preds)}): {preds}")
        print(f"  Successors ({len(succs)}): {succs}")

inspect_button.on_click(on_inspect_click)

display(widgets.HBox([node_selector, inspect_button]))
display(inspect_output)

Output()

### Equivalence Class Browser

In [ ]:
# Equivalence class browser
class_type_dropdown = widgets.Dropdown(
    options=['L-equivalence', 'M-equivalence'],
    value='L-equivalence',
    description='Class type:'
)

browse_button = widgets.Button(description='Show Classes', button_style='info')

class_output = widgets.Output()

def on_browse_click(b):
    with class_output:
        class_output.clear_output()

        prop_set = prop_list_to_proposition_set(PROPOSITION_LIST)
        if prop_set is None or len(prop_set) == 0:
            print("No propositions in builder!")
            print("Use the Proposition Builder above to add propositions.")
            return

        if class_type_dropdown.value == 'L-equivalence':
            classes = L_reorganize(prop_set)
            print(f"L-EQUIVALENCE CLASSES ({len(classes)} classes)")
            print("=" * 60)
            for i, (L_class, M_classes) in enumerate(classes):
                antichain_v, antichain_f = L_class
                print(f"\nClass {i}:")
                print(f"  V-antichain: {', '.join([s.to_string() for s in antichain_v])}")
                print(f"  F-antichain: {', '.join([s.to_string() for s in antichain_f])}")
                print(f"  Members: {len(M_classes)} M-classes")
        else:
            classes = M_reorganize(prop_set)
            print(f"M-EQUIVALENCE CLASSES ({len(classes)} classes)")
            print("=" * 60)
            for i, (M_class, L_classes) in enumerate(classes):
                top_v, top_f = M_class
                v_str = top_v.to_string() if top_v else "(null)"
                f_str = top_f.to_string() if top_f else "(null)"
                print(f"\nClass {i}:")
                print(f"  V-top: {v_str}")
                print(f"  F-top: {f_str}")
                print(f"  Members: {len(L_classes)} L-classes")

browse_button.on_click(on_browse_click)

display(widgets.HBox([class_type_dropdown, browse_button]))
display(class_output)

Output()

## 6. Export

Export your results in various formats.

In [ ]:
# Export controls
export_type_dropdown = widgets.Dropdown(
    options=['CSV (proposition list)', 'DOT (GraphViz)', 'HTML (interactive)'],
    value='CSV (proposition list)',
    description='Format:'
)

filename_text = widgets.Text(
    value='truthmaker_export',
    description='Filename:',
    style={'description_width': '100px'}
)

export_button = widgets.Button(
    description='Export',
    button_style='warning',
    icon='download'
)

export_output = widgets.Output()

def on_export_click(b):
    with export_output:
        export_output.clear_output()
        
        if poset is None:
            print("✗ Please build poset first!")
            return
        
        base_filename = filename_text.value
        export_type = export_type_dropdown.value
        
        try:
            if export_type == 'CSV (proposition list)':
                filename = f"{base_filename}.csv"
                export_to_csv(poset, filename)
                print(f"✓ Exported to {filename}")
            
            elif export_type == 'DOT (GraphViz)':
                filename = f"{base_filename}.dot"
                export_to_dot(poset, filename)
                print(f"✓ Exported to {filename}")
            
            elif export_type == 'HTML (interactive)':
                filename = f"{base_filename}.html"
                fig = create_plotly_graph(poset)
                fig.write_html(filename)
                print(f"✓ Exported to {filename}")
        
        except Exception as e:
            print(f"✗ Export failed: {e}")

export_button.on_click(on_export_click)

display(widgets.VBox([
    export_type_dropdown,
    filename_text,
    export_button
]))
display(export_output)

## 7. Quick Start Example

Run this cell for a quick end-to-end demo with atoms1 (the original 3 mutually exclusive atoms).

In [ ]:
# Quick demo
print("Running quick demo with atoms1...\n")

# Parse atoms1 (using comma format)
demo_props = parse_with_commas(['r;r|g,b;gb', 'g;g|r,b;rb', 'b;b|r,g;rg'])
print(f"Initial: {len(demo_props)} propositions")

# Close
demo_props.close(verbose=False)
print(f"After closure: {len(demo_props)} propositions")

# Build poset
demo_poset = TruthmakerPoset(demo_props, order_type='conjunction')
demo_poset = demo_poset.transitive_reduction()
print(f"Poset: {demo_poset.graph.number_of_edges()} edges\n")

# Visualize
print("Generating visualization...")
demo_fig = create_plotly_graph(demo_poset, layout='hierarchical', color_by='generation')
display(demo_fig)

print("\n✓ Demo complete! Use the cells above for interactive exploration.")